In [16]:
import subprocess
import math
import openai
import glob
from pydub import AudioSegment


def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        # input
        "-i",
        video_path,
        # ignore the vedio => vedio nope => vn
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


# chunk_size <= minutes to cut
def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]

        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")


def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    # find mp3 files in the folder
    for file in files:
        # get transcript // append transcribe to destination
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = openai.Audio.transcribe(
                "whisper-1",
                audio_file,
            )
            # add it to the destination
            text_file.append(transcript["text"])

In [17]:
extract_audio_from_video(
    "./files/DP_Longest_Common_Subsequence_Leetcode1143.mp4", "./files/"
)


cut_audio_in_chunks(

    "./files/DP_Longest_Common_Subsequence_Leetcode1143.mp3", 10, "./files/chunks"

)

transcribe_chunks("./files/chunks", "./files/chunks/transcript.txt")